In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [19]:
# convolution 연산 3->16->32->64

class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        
        self.conv1 = torch.nn.Conv2d(3, 16, 3, stride=1, padding=1)
        self.conv2 = torch.nn.Conv2d(16, 32, 3, stride=1, padding=1)
        self.conv3 = torch.nn.Conv2d(32, 64, 3, stride=1, padding=1)
        
        self.pool = torch.nn.MaxPool2d(2,2)
        
        self.fc1 = torch.nn.Linear(4096, 512)  # conv 연산을 3번 진행 하면서 pool링 과정을 3번 거치면 64,64 -> 8,8로 축소된다.\
        self.fc2 = torch.nn.Linear(512, 128)
        self.fc3 = torch.nn.Linear(128, 33)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25,training=self.training)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25,training=self.training)
        
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25,training=self.training)
        
        x = x.view(-1, 4096)  
        
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc3(x)
        x = F.log_softmax(x, dim=1)
        
        return x

In [20]:
DEVICE = 'cuda'
model_cnn = CNN().to(DEVICE)

In [21]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [22]:
# 이미지 크기 조정 (64,64) , 이미지를 텐서 형태로 변환하고 0~1로 정규화
transform_base = transforms.Compose([transforms.Resize((64,64)),transforms.ToTensor()])

test_dataset = ImageFolder(root='./splited/test/', transform=transform_base)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256,shuffle=True, num_workers=4)

#test 데이터 형식 (data,target)

In [23]:
baseline = torch.load('baseline.pt')
baseline.eval()
test_loss, test_accuracy = evaluate(baseline,test_loader)

print('baseline test acc : ', test_accuracy)

baseline test acc :  88.6218550506947
